In [11]:
from postprocess_base import read_logs
import altair as alt
from glob import glob

runs = read_logs(glob('jeremy/*blocked*.txt'))
runs['FE_nodes_per_compute_node'] = runs['num_unknowns'] / (runs['num_procs'] / runs['num_procs_node']) / runs['dof_per_node']
runs.head()

,file,backend,backend_memtype,hostname,test,num_procs,num_procs_node,degree,quadrature_pts,code,bp,case,num_unknowns,num_elem,dof_per_node,ksp_its,time_per_it,cg_iteration_dps,FE_nodes_per_compute_node
0,jeremy/noether-64-blocked.txt,/cpu/self/xsmm/blocked,host,noether,PETSc CEED Benchmark Problem 1,64,64,1,3,libCEED,1,scalar,42735,41088,1,20,0.000134,317777000.0,42735.0
1,jeremy/noether-64-blocked.txt,/cpu/self/xsmm/blocked,host,noether,PETSc CEED Benchmark Problem 2,64,64,1,3,libCEED,2,vector,128205,41088,3,20,0.000211,608620000.0,42735.0
2,jeremy/noether-64-blocked.txt,/cpu/self/xsmm/blocked,host,noether,PETSc CEED Benchmark Problem 3,64,64,1,3,libCEED,3,scalar,35805,41088,1,1,0.000231,154937000.0,35805.0
3,jeremy/noether-64-blocked.txt,/cpu/self/xsmm/blocked,host,noether,PETSc CEED Benchmark Problem 4,64,64,1,3,libCEED,4,vector,107415,41088,3,1,0.000395,271600000.0,35805.0
4,jeremy/noether-64-blocked.txt,/cpu/self/xsmm/blocked,host,noether,PETSc CEED Benchmark Problem 1,64,64,1,3,libCEED,1,scalar,84035,78016,1,20,0.000225,373093000.0,84035.0


In [12]:
highlight = alt.selection_single(
    on='mouseover',
    fields=['degree', 'time_per_it', 'backend'],
    nearest=True,
    empty='none',
)

bps_select = alt.selection_single(
    fields=['bp'],
)

base = alt.Chart(runs).encode(
    alt.Y('mdofs:Q', title='MDoF/s per CG iteration'),
    alt.Color('degree:N'),
    alt.Size('num_unknowns', scale=alt.Scale(type='log', domain=(1e3, 1e6))),
    alt.Shape('bp:N'),
    tooltip=('bp', 'num_procs', 'backend', 'num_elem', 'degree', 'num_unknowns', 'file'),
).transform_filter(
    bps_select,
).transform_calculate(
    mdofs='datum.cg_iteration_dps/1e6',
)

points = base.mark_point().encode(
    opacity=alt.condition(highlight, alt.value(1), alt.value(.5)),
).add_selection(
    highlight,
)

lines = base.mark_line().encode(
    size=alt.condition(alt.datum.degree - highlight.degree == 0, alt.value(2), alt.value(1))
)

pane = points + lines

composite = (
    pane.encode(
        alt.X('time_per_it', scale=alt.Scale(type='log')),
    ) |
    pane.encode(
        alt.X('FE_nodes_per_compute_node', scale=alt.Scale(type='log', domain=(3e4, 1e7), clamp=True)),
    )
)

activator = alt.Chart(runs).mark_point().encode(
    alt.Y('bp', title='BP'),
    alt.Shape('bp')
).add_selection(bps_select).properties(title='Selection')

activator | composite.properties(title='Noether (2x EPYC 7452), gcc-10')

alt.HConcatChart(...)